# **🏠 네이버 부동산 크롤링**
> 크롤링한 값들 중에 검색이 되지 않은 값을 직접 검색하기 위해 나열

## Contents
- Library Import
- Data Load
- Crawling of Naver real estate home page

## 1. Library Import
- 필요한 라이브러리를 불러옵니다.

In [1]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

## 2. Data Load

In [5]:
apt_info = pd.read_csv('../Data/아파트정보.csv')
apt_area = pd.read_csv('../Data/아파트면적정보.csv')

apt_info2 = pd.read_csv('../Data/아파트정보2.csv')
apt_area2 = pd.read_csv('../Data/아파트면적정보2.csv')


In [18]:
ai = pd.concat([apt_info, apt_info2])
aa = pd.concat([apt_area, apt_area2])

In [8]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
train = pd.read_csv('../Data/KOREAN_train4.csv')
test = pd.read_csv('../Data/KOREAN_test4.csv')

In [9]:
train = train[train['계약년월'] >= 201701]
train = train[train['target'] > 6111.11]
train[['연', '월']] = train['계약년월'].apply(lambda x: list(divmod(x, 100))).tolist()

# 정규표현식 패턴 생성
excluded_apts = ['AirPalace', 'SRvill', '경동팰리스힐', '대길B', '썬앤빌', '재선주택', '코원']

# go
pattern = '|'.join(excluded_apts)

# '열이름' 열에서 패턴이 있는지 확인하여 새로운 열 추가
train['is_아파트'] = np.where(train['아파트명'].str.contains(pattern), 1, 0)

train = train[train['is_아파트'] == 0]

train = train.drop(columns='is_아파트')

In [11]:
# 시구동으로 시군구를 나누기
train[['시', '구', '동']] = train['시군구'].str.split(expand = True)
test[['시', '구', '동']] = test['시군구'].str.split(expand = True)

In [12]:
# 겹치는 동, 아파트명 제거
dap = train[['동', '아파트명']].drop_duplicates(['아파트명']).reset_index()
dap2 = test[['동', '아파트명']].drop_duplicates(['아파트명']).reset_index()

In [13]:
# 동 아파트명 생성하기
dap['동아파트명'] = dap['동'] + ' ' + dap['아파트명']
dap2['동아파트명'] = dap2['동'] + ' ' + dap2['아파트명']

In [14]:
# train, test에 있는 동아파트명 
dap_merge = pd.concat([dap['동아파트명'], dap2['동아파트명']], axis = 0)

In [15]:
# 동아파트명 겹치지 않게 뽑기
d = dap_merge.reset_index()['동아파트명']

In [34]:
# 검색 안된 동아파트명 가져오기
not_searched = ai['동아파트명'][ai['세대수'].isna()]

In [37]:
# 동아파트명 새로운 리스트로 불러오기
new_list = pd.concat([not_searched, d[~d.isin(ai['동아파트명'])]]).reset_index()['동아파트명']

In [38]:
# 저장
new_list.to_csv('../Data/검색안된아파트.csv', index = False)